In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
from collections import Counter

# Lecture des fichiers 

In [2]:
data_dir = "GenerationTest/"

In [3]:
list_folder = os.listdir(data_dir)
num_folder = len(list_folder)
number_of_folder = len(list_folder)

# Cleaning 

In [4]:
to_delete_bdd = {'\n'}
def cleaning_bdd(text):
    for char in text:
        if char in to_delete_bdd:
            text = text.replace(char, ' ')
    return text 

# Creation des listes 

In [5]:
from tqdm import tqdm

In [6]:
bdd = []
sbt = []

for i  in tqdm (range(1000), desc="loading..."):
    path_iteration = data_dir+'/Dir_Fonction_'+str(i+1)
    bdd_path = path_iteration+"/BDD_Fonction_"+str(i+1)+".txt"
    stb_path = path_iteration+"/SBT_Generation_Output/Fonction_"+str(i+1)+"_SBT.txt"
    bdd_txt = open(bdd_path,'r').read()
    sbt_txt = open(stb_path,'r').read()
    clean_bdd = cleaning_bdd(bdd_txt)
    bdd.append(clean_bdd)
    sbt.append(sbt_txt)

loading...: 100%|████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1154.30it/s]


# Recuperation des valeurs max de bdd et sbt

In [7]:
def return_max_len( text ):
    all_len = []
    for unique_code in text:
        code_split = unique_code.split(' ')
        all_len.append(len(code_split))
    return max(all_len)

In [8]:
bdd_counter =Counter([word for sentence in bdd for word in sentence.split(' ')])
sbt_counter =Counter([word for sentence in sbt for word in sentence.split(' ')])

In [9]:
def return_common_words(counter):
    common_words = []
    for word in counter:
        occurence = counter[word]
        if occurence > 700:
            common_words.append(word)
    return common_words

In [10]:
common_words_bdd = return_common_words(bdd_counter)
common_words_sbt = return_common_words(sbt_counter)
common_words_bdd.append('<oov>')
common_words_sbt.append('<oov>')
#common_words_bdd.append(' ')
common_words_sbt.append(' ')

print(common_words_bdd)
print('\n')
print(common_words_sbt)

['Feature:', 'Check', 'the', '', '#', 'result', 'to', 'True', 'Scenario:', 'Given', '=', 'and', 'When', 'i', 'call', 'Then', 'should', 'be', 'False', '<oov>']


['', '(', 'MethodDeclaration', 'Modifier', 'public', ')', 'static', 'SimpleName', 'Parameter', 'PrimitiveType', 'int', 'boolean', 'BlockStmt', 'ExpressionStmt', 'VariableDeclarationExpr', 'VariableDeclarator', 'IntegerLiteralExpr', 'res', 'BinaryExpr', 'NameExpr', 'IfStmt', 'ReturnStmt', 'BooleanLiteralExpr', 'true', 'false', 'UnaryExpr', '<oov>', ' ']


In [11]:
word_to_ind_sbt = {word : ind for ind,word in enumerate(common_words_sbt)}
ind_to_word_sbt = {ind : word for ind,word in enumerate(common_words_sbt)}

word_to_ind_bdd = {word : ind for ind,word in enumerate(common_words_bdd)}
ind_to_word_bdd = {ind : word for ind,word in enumerate(common_words_bdd)}


# Variables d'encodage

In [12]:
number_folder = len(bdd)
max_words_sbt = return_max_len(sbt)
max_words_bdd = return_max_len(bdd)
number_of_words_sbt = len(common_words_sbt)
number_of_words_bdd = len(common_words_bdd)

print(' number of forlder :',number_folder)
print(' number maximum of word in one sbt code :',max_words_sbt)
print(' number maximum of word in one bdd code :',max_words_bdd)
print(' number of different word in sbt :',number_of_words_sbt)
print(' number of different word in bdd :',number_of_words_bdd)

 number of forlder : 1000
 number maximum of word in one sbt code : 255
 number maximum of word in one bdd code : 92
 number of different word in sbt : 28
 number of different word in bdd : 20


# Encodage du BDD, en one hot encoding

In [13]:
encoded_bdd = np.zeros((number_folder,max_words_bdd,number_of_words_bdd))
encoded_bdd.shape

(1000, 92, 20)

In [14]:
variables_bdd = []
for i,sentence in  enumerate(bdd):
    bdd_split = sentence.split( ' ')
    for j,word in enumerate (bdd_split):
        if word in common_words_bdd:
            encoded_bdd[i,j,word_to_ind_bdd[word]] = 1.
        else:
            variables_bdd.append(word)
            encoded_bdd[i,j,word_to_ind_bdd['<oov>']] = 1.

# Encodage des variables en onehot

Recuperation taille max des variables 

In [15]:
max_len_variable_bdd = max([len(x) for x in variables_bdd])
max_len_variable_bdd

19

Creation d'un vocabulaire pour les variables 

In [76]:
vocab_variables_bdd = {l for char in variables_bdd for l in char}
vocab_variables_bdd.add('<void>')

In [78]:
char_to_ind_bdd = {char : ind for ind,char in enumerate(vocab_variables_bdd)}
ind_to_char_bdd = {ind : char for ind,char in enumerate(vocab_variables_bdd)}

Nombre de caractere differents dans le vocabulaire des variables 

In [79]:
number_char_variables_bdd = len(vocab_variables_bdd)
print(number_char_variables_bdd)

40


nombre de variables par fichier bdd

In [80]:
number_variables_bdd = 13

In [81]:
encoded_variables_bdd = np.zeros((number_folder,number_variables_bdd, max_len_variable_bdd, number_char_variables_bdd))
encoded_variables_bdd.shape

(1000, 13, 19, 40)

Encodage des variables

In [182]:
for i,sentence in  enumerate(bdd):
    bdd_split = sentence.split( ' ')
    x = 0 # compteur qui calcule le nombre de mots hors du vocab rencontrés 

    for j,word in enumerate (bdd_split):
        if word not in common_words_bdd:
            for h,char in enumerate ( word ):
                encoded_variables_bdd[i,x,h,char_to_ind_bdd[char]] = 1.
            x=x+1

In [183]:
encoded_variables_bdd_2d = np.reshape(encoded_variables_bdd,
                        (number_folder, number_variables_bdd*max_len_variable_bdd,number_char_variables_bdd))

In [184]:
encoded_variables_bdd_2d.shape

(1000, 247, 40)

In [129]:
encoded_variables_bdd_2d[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

# Decodage des variables en onehot

In [89]:
def decode_var_bdd ( encoded_variables):
    decoded_variables = []
    encoded_variables_test = encoded_variables
    for variables in encoded_variables_test:
        for chars in variables:
            for i,char in enumerate (chars):
                if char == 1:
                    decoded_variables.append(ind_to_char_bdd[i])
        decoded_variables.append('--')
    decoded_sequence = ''.join(decoded_variables)
    return decoded_sequence

In [90]:
test_decode = decode_var_bdd(encoded_variables_bdd[0])
test_decode

'Fonction_1--bdd_Fonction_1_1--z---6--e--11--Fonction_1--bdd_Fonction_1_2--z--3--e--3--Fonction_1--'

# Encodage des variables sparse

In [130]:
encoded_variables_bdd_sparse = np.full((number_folder,number_variables_bdd, max_len_variable_bdd),char_to_ind_bdd['<void>'])
encoded_variables_bdd_sparse.shape

(1000, 13, 19)

In [134]:
for i,sentence in enumerate(bdd):
    bdd_split = sentence.split(' ')
    x=0
    for j,word in enumerate (bdd_split):
        if word not in common_words_bdd:
            for h,char in enumerate(word):
                encoded_variables_bdd_sparse[i,x,h]=char_to_ind_bdd[char]
            x=x+1

# Decodage des variables sparse 

In [139]:
def decode_var_bdd_sparse( encoded_var_bdd_sparse):
    decoded_var_bdd_sparse = []
    for i,chars in enumerate ( encoded_var_bdd_sparse):
        for char in chars:
            if char == char_to_ind_bdd['<void>']:
                break
            else :
                decoded_var_bdd_sparse.append(ind_to_char_bdd[char])
        decoded_var_bdd_sparse.append('--') 
    decoded_var_bdd_sparse = ''.join(decoded_var_bdd_sparse)
    return decoded_var_bdd_sparse

In [140]:
decode_var_bdd_sparse(encoded_variables_bdd_sparse[0])

'Fonction_1--bdd_Fonction_1_1--z---6--e--11--Fonction_1--bdd_Fonction_1_2--z--3--e--3--Fonction_1--'

# Decodage du BDD

In [97]:
def decode_bdd(encoded):
    decoded_sequence_bdd = []
    for i,words in enumerate(encoded):
        
        for j,word in enumerate(words):
            
            if (word==1):
                decoded_sequence_bdd.append(ind_to_word_bdd[j])
                decoded_sequence_bdd.append(' ')
        
    decoded_sequence_bdd = ''.join(decoded_sequence_bdd)
    return decoded_sequence_bdd

In [98]:
decod = decode_bdd(encoded_bdd[0])
decod

'Feature: Check the <oov>     # Check the result to True   Scenario: <oov>     Given <oov> = <oov> and <oov> = <oov>     When i call <oov>     Then the result should be True    # Check the result to False   Scenario: <oov>     Given <oov> = <oov> and <oov> = <oov>     When i call <oov>     Then the result should be False  '

# ---------------------------------------------------------------------------------------------------------------
# Encodage du SBT

In [99]:
encoded_sbt = np.full((number_folder,max_words_sbt), word_to_ind_sbt[' '])

In [100]:
variables_sbt = []
for i,sentence in enumerate(sbt):
    sbt_split = sentence.split(' ')
    for j,word in enumerate (sbt_split):
        if word in common_words_sbt:
            encoded_sbt[i,j]=word_to_ind_sbt[word]
        else :
            variables_sbt.append(word)
            encoded_sbt[i,j]=word_to_ind_sbt['<oov>']

# Encodage des variables 

Test envisageable : normaliser toutes les valeurs pour qu'elles soient comprises entre 0 et 1 

In [107]:
number_variables_sbt = 22

In [108]:
max_len_variable_sbt = max([len(x) for x in variables_sbt])
max_len_variable_sbt

13

In [114]:
vocab_variables_sbt = {l for char in variables_bdd for l in char}
vocab_variables_sbt.add('<void>')

In [115]:
char_to_ind_sbt = {char : ind for ind,char in enumerate(vocab_variables_sbt)}
ind_to_char_sbt = {ind : char for ind,char in enumerate(vocab_variables_sbt)}

In [119]:
number_char_sbt = len(vocab_variables_sbt)
number_char_sbt

40

In [120]:
encoded_variables_sbt = np.full((number_folder,number_variables_sbt, max_len_variable_sbt),char_to_ind_sbt['<void>'])
encoded_variables_sbt.shape

(1000, 22, 13)

In [121]:
for i,sentence in enumerate(sbt):
    sbt_split = sentence.split(' ')
    x=0
    for j,word in enumerate (sbt_split):
        if word not in common_words_sbt:
            for h,char in enumerate(word):
                encoded_variables_sbt[i,x,h]=char_to_ind_sbt[char]
            x=x+1

# Encodage en 2d

In [153]:
encoded_variables_sbt_2d = np.full((number_folder,number_variables_sbt* max_len_variable_sbt),char_to_ind_sbt['<void>'])
encoded_variables_sbt_2d.shape

(1000, 286)

In [154]:
for i,sentence in enumerate(sbt):
    sbt_split = sentence.split(' ')
    x=0
    for j,word in enumerate (sbt_split):
        if word not in common_words_sbt:
            for h,char in enumerate(word):
                encoded_variables_sbt_2d[i,x*13+h]=char_to_ind_sbt[char]
            x=x+1

In [155]:
encoded_variables_sbt_2d_reshaped= np.reshape(encoded_variables_sbt_2d,
                                              (number_folder,number_variables_sbt,max_len_variable_sbt))
print(encoded_variables_sbt_2d_reshaped.shape)
print(encoded_variables_sbt_2d.shape)

(1000, 22, 13)
(1000, 286)


# Decodage des variables 

In [159]:
def decode_var_sbt( encoded_var_sbt):
    decoded_var_sbt = []
    x=0
    for i,chars in enumerate ( encoded_var_sbt):
        for char in chars:
            if char == char_to_ind_sbt['<void>']:
                x=x+1
                break
            else :
                decoded_var_sbt.append(ind_to_char_sbt[char])
        decoded_var_sbt.append('--') 
    decoded_var_sbt = ''.join(decoded_var_sbt)
    return decoded_var_sbt

In [160]:
decode_var_sbt(encoded_variables_sbt[0])

'Fonction_1--Fonction_1--z--z--e--e--s--s--6--6--c--c--0--0--z--z--e--e--s--s--c--c--'

In [164]:
decode_var_sbt(encoded_variables_sbt_2d_reshaped[0])

'Fonction_1--Fonction_1--z--z--e--e--s--s--6--6--c--c--0--0--z--z--e--e--s--s--c--c--'

# Decodage SBT

In [165]:
def decode_sbt(encoded):
    decoded = []
    for word in encoded:
        decoded.append(ind_to_word_sbt[word])
        decoded.append(' ')
        
    decoded = ''.join(decoded)
    return decoded


# Repartition test train

In [166]:
from sklearn.model_selection import train_test_split

# Template

In [167]:
X_train, X_test, y_train, y_test = train_test_split( encoded_sbt, encoded_bdd, test_size=0.1, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(900, 255)
(100, 255)
(900, 92, 20)
(100, 92, 20)


# Variables 

In [203]:
X_var_train, X_var_test, y_var_train, y_var_test = train_test_split( encoded_variables_sbt_2d,
                                                                    encoded_variables_bdd_sparse,
                                                                    test_size=0.1, random_state=42)

In [204]:
encoded_variables_bdd_sparse.shape

(1000, 13, 19)

In [205]:
encoded_variables_sbt_2d.shape

(1000, 286)

In [206]:
print(X_var_train.shape)
print(y_var_train.shape)
print(X_var_test.shape)
print(y_var_test.shape)

(900, 286)
(900, 13, 19)
(100, 286)
(100, 13, 19)


# Creation du Modele

In [197]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, TimeDistributed, LSTM, Bidirectional, Embedding, RepeatVector
from tensorflow.keras.losses import sparse_categorical_crossentropy

# Template Model

In [198]:
model = Sequential()
model.add(Embedding(number_of_words_sbt, 256, input_length=max_words_sbt, mask_zero=True))
model.add(LSTM(256))
model.add(RepeatVector(max_words_bdd))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(number_of_words_bdd, activation='softmax')))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [199]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 255, 256)          7168      
_________________________________________________________________
lstm_20 (LSTM)               (None, 256)               525312    
_________________________________________________________________
repeat_vector_10 (RepeatVect (None, 92, 256)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 92, 256)           525312    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 92, 20)            5140      
Total params: 1,062,932
Trainable params: 1,062,932
Non-trainable params: 0
_________________________________________________________________


# Variables Model

In [207]:
model_var = Sequential()
model_var.add(Embedding(number_char_sbt, 256, input_length=number_variables_sbt* max_len_variable_sbt, mask_zero=True))
model_var.add(LSTM(256))
model_var.add(RepeatVector(13))
model_var.add(LSTM(256, return_sequences=True))
model_var.add(TimeDistributed(Dense(19, activation='softmax')))

model_var.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [201]:
model_var.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 286, 256)          10240     
_________________________________________________________________
lstm_22 (LSTM)               (None, 256)               525312    
_________________________________________________________________
repeat_vector_11 (RepeatVect (None, 247, 256)          0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 247, 256)          525312    
_________________________________________________________________
time_distributed_11 (TimeDis (None, 247, 40)           10280     
Total params: 1,071,144
Trainable params: 1,071,144
Non-trainable params: 0
_________________________________________________________________


In [202]:
model_var.fit(X_var_train, y_var_train ,batch_size = 16, epochs=10, validation_data = (X_var_test, y_var_test))

Epoch 1/10


InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [3952,40] and labels shape [158080]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-202-fc3c5f83dba9>:1) ]] [Op:__inference_train_function_72335]

Function call stack:
train_function


# Test

In [102]:
for n in range (10):
    test_input = X_test[n]
    test_input = np.expand_dims(test_input,axis=0)
    prediction = model.predict(test_input)[0]

    #print(prediction.shape)
    sequence_prediction = []
    for pred in (prediction):
        sequence_prediction.append(np.argmax(pred))
    decoded_sequence = []
    for word in sequence_prediction:
        decoded_sequence.append(ind_to_word_bdd[word])
    decoded_sequence.append(' ')

    decoded_sequence = ''.join(decoded_sequence)
    print('Predicted BDD  ',n+1,' :')
    print('\n')
    print(decoded_sequence)
    print('\n')
    print('Expected BDD ',n+1,' :')
    print('\n')
    print(decode_bdd(y_test[n]))
    print('\n')

Predicted BDD   1  :


Feature:Checkthe<oov>#ChecktheresulttoTrueScenario:Scenario:<oov><oov><oov><oov><oov><oov><oov>iicallthetheshouldbe#ChecktheresulttoFalse<oov><oov><oov><oov><oov><oov><oov>icallthethetheshouldbe 


Expected BDD  1  :


Feature: Check the <oov>     # Check the result to True   Scenario: <oov>     Given <oov> = <oov> and <oov> = <oov>     When i call <oov>     Then the result should be True    # Check the result to False   Scenario: <oov>     Given <oov> = <oov> and <oov> = <oov>     When i call <oov>     Then the result should be False  


Predicted BDD   2  :


Feature:Checkthe<oov>#ChecktheresulttoTrueScenario:Scenario:<oov><oov><oov><oov><oov><oov><oov>iicallthetheshouldbe#ChecktheresulttoFalse<oov><oov><oov><oov><oov><oov><oov>icallthethetheshouldbe 


Expected BDD  2  :


Feature: Check the <oov>     # Check the result to True   Scenario: <oov>     Given <oov> = <oov> and <oov> = <oov>     When i call <oov>     Then the result should be True    # Check the re

In [103]:
test_input = X_test[0]
test_input

array([ 0,  1,  2,  1,  3,  4,  0,  5,  3,  4,  0,  1,  3,  6,  0,  5,  3,
        6,  0,  1,  7, 26,  5,  7, 26,  1,  8,  1,  9, 10,  5,  9, 10,  1,
        7, 26,  5,  7, 26,  5,  8,  1,  8,  1,  9, 10,  5,  9, 10,  1,  7,
       26,  5,  7, 26,  5,  8,  1,  9, 11,  5,  9, 11,  1, 12,  1, 13,  1,
       14,  1, 15,  1,  9, 10,  5,  9, 10,  1,  7, 26,  5,  7, 26,  1, 25,
        1, 16, 26,  5, 16, 26,  5, 25,  5, 15,  5, 14,  5, 13,  1, 13,  1,
       14,  1, 15,  1,  9, 10,  5,  9, 10,  1,  7, 26,  5,  7, 26,  1, 25,
        1, 16, 26,  5, 16, 26,  5, 25,  5, 15,  5, 14,  5, 13,  1, 13,  1,
       14,  1, 15,  1,  9, 10,  5,  9, 10,  1,  7, 17,  5,  7, 17,  1, 18,
        1, 18,  1, 19,  1,  7, 26,  5,  7, 26,  5, 19,  1, 19,  1,  7, 26,
        5,  7, 26,  5, 19,  5, 18,  1, 19,  1,  7, 26,  5,  7, 26,  5, 19,
        5, 18,  5, 15,  5, 14,  5, 13,  1, 20,  1, 18,  1, 19,  1,  7, 17,
        5,  7, 17,  5, 19,  1, 19,  1,  7, 26,  5,  7, 26,  5, 19,  5, 18,
        1, 12,  1, 21,  1

In [104]:
test_input = np.expand_dims(test_input,axis=0)
prediction = model.predict(test_input)[0]

In [105]:
prediction[0]

array([0.55079305, 0.20506339, 0.05037852, 0.02406569, 0.00592632,
       0.00389126, 0.00325892, 0.01044804, 0.00827105, 0.00495665,
       0.00685369, 0.00671526, 0.01215183, 0.00979984, 0.0169343 ,
       0.00976743, 0.00803564, 0.01289711, 0.01399474, 0.03579716],
      dtype=float32)